## Limited wildcard

[BoardGameGeek forum post.](https://boardgamegeek.com/thread/869326/article/40240589#40240589)

* Start with some number of wildcards ("W").
  * A wildcard can stand in for some other cards but not others. For example, it might be able to stand in for "C", "D", or "E" but not "A" or "B".
* Draw some cards from a deck.
* What's the chance that you will be able to make a desired multiset of cards?

In [1]:
import piplite
await piplite.install("icepool")

import icepool
import time

class EvalWildcard(icepool.MultisetEvaluator):
    def next_state_ascending(self, state, outcome, target, *counts):
        # state = the number of wildcards needed.
        if state is None:
            state = 0
        total_count = sum(counts)
        # Final: wildcards.
        if outcome == 'W':
            if state == 'fail':
                return False
            else:
                return total_count >= state
        if state == 'fail':
            return state
        # Could potentially use wildcards.
        if outcome >= 'C':
            return state + max(target - total_count, 0)
        # Ineligible for wildcards.
        if total_count < target:
            return 'fail'
        return state

    def extra_outcomes(self, *_):
        # Always process wildcard.
        return ('W',)

# When expressed as a sequence, each appearance counts as one card.
target = list('ABBCDDEEE')
# When expressed as a dict, the value gives the number of cards.
deal = icepool.Deck({'A': 3, 'B': 8, 'C': 2, 'D': 3, 'E': 3, 'F': 113, 'W': 7}).deal(35)
hand = list('W')

evaluator = EvalWildcard()

start_ns = time.perf_counter_ns()

# The counts resulting from the three arguments are supplied as
# the last three arguments to next_state.
result = evaluator.evaluate(target, deal, hand)

end_ns = time.perf_counter_ns()
elapsed_ms = (end_ns - start_ns) * 1e-6
print(f'Computation time: {elapsed_ms:0.1f} ms')

print(f'{result:md:o|q==|%==}')

Computation time: 10.4 ms
Die with denominator 903552809026475628595978062186780

| Outcome |                          Quantity | Probability |
|:--------|----------------------------------:|------------:|
| False   | 816069609857287121983676726542308 |  90.317865% |
| True    |  87483199169188506612301335644472 |   9.682135% |


